In [1]:
from faster_whisper import WhisperModel, BatchedInferencePipeline

c:\Users\AKM\Desktop\AUXMET\auxmet-bot\.venv\Lib\site-packages\ctranslate2\__init__.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
c:\Users\AKM\Desktop\AUXMET\auxmet-bot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch 
torch.cuda.is_available()

True

In [ ]:
from groq import Groq
groq_client = Groq(api_key="")

def transcribe_audio(file_path: str) -> str:
    with open(file_path, "rb") as f:
        response = groq_client.audio.transcriptions.create(
            model="whisper-large-v3-turbo",
            file=f,
            response_format="text"
        )
    return response

In [28]:
res = transcribe_audio("C:/Users/AKM/Desktop/AUXMET/auxmet-bot/0.wav")

In [29]:
print(res)

 Hello, Aditya. Can you explain what is overfitting in the context of machine learning models? That's great. Text-to-speech, TTS. Synthesis has seen significant advancements in recent years, with numerous applications such as virtual assistants, audiobooks, and voiceover narration benefiting from increasingly natural and expressive synthetic speech one. Two. Some previous works have made significant progress towards human-level performance three. Four. Five.


In [14]:
model = WhisperModel(
                model_size_or_path="large-v3",
                device="cuda",
                compute_type="int8",
            )
pipeline = BatchedInferencePipeline(model=model)

In [27]:
segs,_ = pipeline.transcribe(
                    "C:/Users/AKM/Desktop/AUXMET/auxmet-bot/0.wav",
                    language="en",
                    # batch_size=4,
                    vad_filter = True,
                    vad_parameters = {
                "threshold": 0.3,  # it will be ok for low noises
                "min_speech_duration_ms": 150,
                "min_silence_duration_ms": 1000,
                
            },
             word_timestamps = True
    )
text = "".join([seg.text for seg in segs ])
text

" Hello, Aditya. Can you explain what is overfitting in the context of machine learning models? That's great text-to-speech, TTS. Synthesis has seen significant advancements in recent years, with numerous applications such as virtual assistants, audiobooks, and voice-over narration benefiting from increasingly natural and expressive synthetic speech 1. 2. Some previous works have made significant progress towards human-level performance 3. 4. 5."

In [55]:
from kokoro import KPipeline
from IPython.display import display, Audio
import soundfile as sf
import torch
print(torch.cuda.is_available())
pipeline = KPipeline(lang_code='a',device="cuda")


True


c:\Users\AKM\Desktop\AUXMET\auxmet-bot\.venv\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
c:\Users\AKM\Desktop\AUXMET\auxmet-bot\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [26]:
text = """
In this paper, we present StyleTTS 2, a text-to-speech (TTS) model that leverages
style diffusion and adversarial training with large speech language models (SLMs)
to achieve human-level TTS synthesis. StyleTTS 2 differs from its predecessor by
modeling styles as a latent random variable through diffusion models to generate
the most suitable style for the text without requiring reference speech,
"""

generator = pipeline(text, voice='af_heart')
for i, (gs, ps, audio) in enumerate(generator):
    # print(i, gs, ps)
    print(gs)
    
    

In this paper, we present StyleTTS 2, a text-to-speech (TTS) model that leverages
style diffusion and adversarial training with large speech language models (SLMs)
to achieve human-level TTS synthesis. StyleTTS 2 differs from its predecessor by
modeling styles as a latent random variable through diffusion models to generate
the most suitable style for the text without requiring reference speech,


In [1]:
from pydantic import BaseModel,Field
from typing import Dict 
from langchain_groq import ChatGroq 
from urllib.parse import quote_plus
import requests as req 
import os 
from dotenv import load_dotenv 
load_dotenv()

model = ChatGroq(model = "meta-llama/llama-4-maverick-17b-128e-instruct",api_key=os.getenv("GROQ_API"))

In [11]:
# @tool /
def give_links(topics:list):
    qw = {}
    for q in topics:
        query = q.topic +" "+q.subconcept
        search_query = quote_plus(query)

        url = f"https://duckduckgo.com/html/?q={search_query}"

        headers = {
            "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/121.0",
        }
        resp = req.get(url,headers=headers)

        from bs4 import BeautifulSoup 
        soup = BeautifulSoup(resp.text, "html.parser")
        result = [] 

        for a in soup.select(".result__a"):
            if len(result) == 4 :
                break 
            result.append(a["href"])
        qw[query] = {"links":result,"qa":[q.question,q.wrong_answer]} 
    return qw 
        


    

In [118]:
from typing import Any,List
from pydantic import BaseModel, Field

class question_analysis(BaseModel):
    question_asked:str = Field(description="Total question asked from the topic.")
    answered_correct:str = Field(description="Total question answered correct from the topic.")
    question_skip:str  = Field(description="Total question skipped from the topic.")

class WrongQAItem(BaseModel):
    question: str = Field(description="The question text")
    wrong_answer: str = Field(description="The incorrect answer provided")
    topic: str = Field(description="The main topic/concept area this question belongs to")
    subconcept: str = Field(description="The specific subconcept within the topic")

class OutputParse(BaseModel):
    wrong_qa: List[WrongQAItem] = Field(
        description="List of questions with wrong answers, including their topics and subconcepts"
    )

class QuestionAnalysis(BaseModel):
    questions_asked: int = Field(description="Total number of questions asked from this domain/topic")
    Number_of_question: int = Field(description="Number of questions answered correctly from this domain/topic")
    answered_incorrectly: int = Field(description="Number of questions answered incorrectly from this domain/topic")
    questions_skipped: int = Field(description="Number of questions skipped or not answered from this domain/topic")
    partial_answers: int = Field(description="Number of questions with partially correct answers from this domain/topic")

class domain_specific_result(BaseModel):
    domainName: str = Field(description="Name of the domain")
    Number_of_question: int = Field(description="The number of questions that are asked in that domain")
    Number_of_answers_correct: int = Field(description="Number of questions that are correctly answered from particular domain")
    Number_of_answers_incorrect: int = Field(description="Number of questions that are answered incorrectly from particular domain") 
    Number_of_skipped_questions: int = Field(description="Number of questions that are skipped from that particular domain")

class ValidationOutput(BaseModel):
    average_technical_skills_score: float = Field(
        description="Average technical score calculated as: (correct_answers + 0.5*partial_answers) / total_attempted_questions. Range: 0.0 to 1.0",
        ge=0.0, le=1.0
    )
    domain_specific_technical_knowledge_score:List[domain_specific_result] = Field(
        description="List of the Domain-wise technical knowledge evaluation"
    )
    communication_skills_score: float = Field(
        description="Score based on clarity, fluency, articulation, and professional tone. Range: 1.0 to 10.0",
        ge=1.0, le=10.0
    )
    question_understanding_score: float = Field(
        description="Score measuring how well the candidate comprehended and interpreted the questions. Range: 1.0 to 10.0",
        ge=1.0, le=10.0
    )
    problem_solving_approach_score: float = Field(
        description="Score evaluating the candidate's thought process, logical reasoning, and problem-solving methodology. Range: 1.0 to 10.0",
        ge=1.0, le=10.0
    )
    depth_of_knowledge_score: float = Field(
        description="Score measuring the depth and comprehensiveness of answers. Moderate depth: 8-10, Basic depth: 1-7. Range: 1.0 to 10.0",
        ge=1.0, le=10.0
    )
  
query2 = """ 

You are helpful ai. Your task is to validate the the interview conversation and You will be provided with conversation.

You are also provided with the tool that is used to add the conversation of both question asked by ai and answer given by the candidate when candidate gives incorrect answer 
call tool only when the answer is incorrect or candidate skipped the question. and pass that message turn into it.


evaluation parameters : 

- Average techical skills score : Average of the domain specific technical skills  
- domain specific technical knowlage score : score with technical topics in the json format based on correctness of the answer by the candidate in the conversation. If answer is correct 1 other wise 0 
- communication skills : score should be based on clarity,fluency,tone. Score range between 1 to 10 
- questions understanding : How well the questions are understood by the user. Score range between 1 to 10  
- Problem-Solving Approach : Whether the user explains thought process. Score range between 1 to 10 
- DepthOfKnowledge : Candidate answer the question in how much in depth . here if candidate explains the answer in moderate level of depth the socre should range between 8-10 and  if candidate answers less than moderate level then score should range between 1-7.

output example :

technical skills score : 1 is number of correct question answered 3 is total question asked and 1 is number of question skipped so average correct question answered is 1÷2 equals to 0.5 don't add questions that are not answered while calculating the average. 
domain specific technical knowlage score : [
                                    {
                                        domain_name:backend
                                        question_asked:2
                                        answered_correct:1
                                        question_skip:1
                                    }

                                    {
                                    domain_name: machine_leariing
                                    question_asked:1,
                                    answered_correct:1
                                    question_skip:0
                                    }
                                        ]
communication skills score : 7.0
question understanding score : 7.2
Problem-Solving Approach score : 8.4
DepthOfKnowledge score : 5.4 

Conversation :
{context}



"""
query = """
You are an AI assistant that evaluates interview conversations. Analyze the conversation and return evaluation scores in the exact JSON format specified below.

**RETURN ONLY THIS JSON STRUCTURE:**

{{
    "average_technical_skills_score": <float between 0.0 and 1.0>,
    "domain_specific_technical_knowledge_score": [
        {{
            "domainName": "<string>",
            "Number_of_question": <integer>,
            "Number_of_answers_correct": <integer>,
            "Number_of_answers_incorrect": <integer>,
            "Number_of_skiped_questions": <integer>
        }}
    ],
    "communication_skills_score": <float between 1.0 and 10.0>,
    "question_understanding_score": <float between 1.0 and 10.0>,
    "problem_solving_approach_score": <float between 1.0 and 10.0>,
    "depth_of_knowledge_score": <float between 1.0 and 10.0>
}}

**EVALUATION GUIDELINES:**

1. **Average Technical Skills Score** (0.0 - 1.0):
   - Formula: correct_answers ÷ total_attempted_questions
   - Do NOT include skipped questions in denominator
   - Example: 5 correct out of 10 attempted = 0.5

2. **Domain Specific Technical Knowledge Score**:
   - Categorize questions by domain (security, machine_learning, backend, frontend, databases, networking, general_computing, etc.)
   - For EACH domain, count:
     * Number_of_question: Total questions in this domain
     * Number_of_answers_correct: Correct answers
     * Number_of_answers_incorrect: Incorrect answers
     * Number_of_skiped_questions: Unanswered questions

3. **Communication Skills Score** (1.0 - 10.0):
   - Evaluate clarity, fluency, articulation, and professional tone
   - 8-10: Clear, fluent, professional
   - 5-7: Adequate communication with some issues
   - 1-4: Poor communication, unclear, unprofessional

4. **Question Understanding Score** (1.0 - 10.0):
   - Assess how well the candidate understood each question
   - 8-10: Clearly understood questions and addressed them properly
   - 5-7: Understood most questions with minor misinterpretations
   - 1-4: Frequently misunderstood or didn't address questions

5. **Problem-Solving Approach Score** (1.0 - 10.0):
   - Evaluate logical reasoning and thought process explanation
   - 8-10: Clear methodology, logical steps, explains reasoning
   - 5-7: Some logical approach, limited explanation
   - 1-4: No clear methodology, poor reasoning

6. **Depth of Knowledge Score** (1.0 - 10.0):
   - Assess thoroughness and comprehensiveness of answers
   - 8-10: Moderate to deep explanations with context and details
   - 5-7: Basic explanations with some details
   - 1-4: Shallow answers with minimal explanation

**INSTRUCTIONS:**
1. Read each question-answer pair carefully
2. Determine if the answer is technically correct or incorrect
3. Categorize each question by its primary technical domain
4. Count correct/incorrect answers per domain
5. Calculate the average technical score
6. Evaluate soft skills based on overall communication quality
7. Return the results in the exact JSON format above

**CONVERSATION DATA:**
{context}
"""

# Example of how to use it correctly
questions = [
  {
    "Question": "What is SSL/TLS?",
    "Answer": "SSL/TLS are protocols for securing communications over a computer network."
  },
  {
    "Question": "What is SQL injection?",
    "Answer": "SQL injection is a security vulnerability where attackers insert malicious SQL statements into input fields."
  },
  {
    "Question": "What is a batch in machine learning?",
    "Answer": "A batch is a subset of training data processed together in one iteration of model training."
  },
  {
    "Question": "What is transfer learning?",
    "Answer": "Transfer learning is the process of reusing a pre-trained model on a new, related problem."
  },
  {
    "Question": "What is a CDN?",
    "Answer": "A CDN is a system of distributed servers that deliver content to users based on their geographic location."
  },
  {
    "Question": "What is tokenization in NLP?",
    "Answer": "Tokenization is the process of breaking text into smaller units such as words or subwords."
  },
  {
    "Question": "What is a hyperparameter in machine learning?",
    "Answer": "A hyperparameter is a parameter set before the learning process begins, controlling the behavior of the training algorithm."
  },
  {
    "Question": "What is backpropagation?",
    "Answer": "Backpropagation is an algorithm used to calculate the gradient of the loss function with respect to weights in neural networks."
  },
  {
    "Question": "What is Kubernetes?",
    "Answer": "Kubernetes is an open-source platform for automating deployment, scaling, and management of containerized applications."
  },
  {
    "Question": "What is Docker?",
    "Answer": "Docker is a platform for developing, shipping, and running applications inside containers."
  },
  {
    "Question": "What is artificial general intelligence?",
    "Answer": "Artificial general intelligence is a type of AI that can perform any intellectual task a human can do."
  },
  {
    "Question": "What is overfitting in machine learning?",
    "Answer": "Overfitting occurs when a model performs better on new data than on training data."
  },
  {
    "Question": "What is the role of a load balancer in backend architecture?",
    "Answer": "A load balancer distributes incoming traffic to a single server to improve latency."
  },
  {
    "Question": "What is the difference between supervised and unsupervised learning?",
    "Answer": "In supervised learning, data is unlabeled, while in unsupervised learning, data is labeled."
  },
  {
    "Question": "What is gradient descent used for in machine learning?",
    "Answer": "Gradient descent is used to maximize the loss function."
  },
  {
    "Question": "What does CPU stand for?",
    "Answer": "CPU stands for Central Processing Unit."
  },
  {
    "Question": "What is the function of RAM in a computer?",
    "Answer": "RAM permanently stores files and programs even when the computer is turned off."
  },
  {
    "Question": "What is an API?",
    "Answer": "An API is a graphical design of a website."
  },
  {
    "Question": "What is a database index?",
    "Answer": "A database index is a backup of the entire database."
  },
  {
    "Question": "What is the capital of France?",
    "Answer": "The capital of France is Berlin."
  },
  {
    "Question": "What is cloud computing?",
    "Answer": "Cloud computing is a method of physically storing all data in a user’s local computer."
  },
  {
    "Question": "What is Docker used for?",
    "Answer": "Docker is used for creating and managing virtual machines."
  },
  {
    "Question": "What is Kubernetes?",
    "Answer": "Kubernetes is a database management tool."
  },
  {
    "Question": "What is the main function of DNS?",
    "Answer": "DNS is used to encrypt internet traffic."
  },
  {
    "Question": "What is HTTPS?",
    "Answer": "HTTPS is a protocol for transferring hypertext securely over the internet."
  },
  {
    "Question": "What is Python?",
    "Answer": "Python is a type of snake only found in South America."
  },
  {
    "Question": "What is a neural network?",
    "Answer": "A neural network is a network of physical human brain cells connected via cables."
  },
  {
    "Question": "What is SQL used for?",
    "Answer": "SQL is used to design the frontend of a website."
  },
  {
    "Question": "What is NoSQL?",
    "Answer": "NoSQL is a protocol for encrypting emails."
  },
  {
    "Question": "What is HTML?",
    "Answer": "HTML is a machine learning algorithm for data prediction."
  },
  {
    "Question": "What is CSS?",
    "Answer": "CSS is a programming language for server-side logic."
  },
  {
    "Question": "What is JavaScript?",
    "Answer": "JavaScript is a language used only for mobile app backends."
  },
  {
    "Question": "What is a microservice?",
    "Answer": "A microservice is a small computer used in embedded systems."
  },
  {
    "Question": "What is latency?",
    "Answer": "Latency is the speed at which a processor completes all possible tasks instantly."
  },
  {
    "Question": "What is caching?",
    "Answer": "Caching is a process of deleting all stored data from a server."
  },
  {
    "Question": "What is version control?",
    "Answer": "Version control is a method to change the color theme of code editors."
  },
  {
    "Question": "What is Git?",
    "Answer": "Git is a tool for editing images."
  },
  {
    "Question": "What is machine learning?",
    "Answer": "Machine learning is a process of teaching machines to walk and talk like humans physically."
  },
  {
    "Question": "What is deep learning?",
    "Answer": "Deep learning is learning to swim in deep waters."
  },
  {
    "Question": "What is artificial intelligence?",
    "Answer": "Artificial intelligence is the study of making computers conscious beings."
  },
  {
    "Question": "What is computer vision?",
    "Answer": "Computer vision is a method of giving computers the ability to read physical books."
  },
  {
    "Question": "What is reinforcement learning?",
    "Answer": "Reinforcement learning is when a model learns by reading documentation only once."
  },
  {
    "Question": "What is NLP?",
    "Answer": "NLP is the study of network-level protocols."
  },
  {
    "Question": "What is a confusion matrix?",
    "Answer": "A confusion matrix is a table used to store all programming errors in a project."
  },
  {
    "Question": "What is data normalization?",
    "Answer": "Data normalization is a way of storing all data in capital letters."
  },
  {
    "Question": "What is feature scaling?",
    "Answer": "Feature scaling is a method to change the scale of graphs to match your monitor size."
  },
  {
    "Question": "What is a REST API?",
    "Answer": "A REST API is a service used for putting a computer to sleep remotely."
  },
  {
    "Question": "What is GraphQL?",
    "Answer": "GraphQL is a method for encrypting graph data using quantum physics."
  },
  {
    "Question": "What is load testing?",
    "Answer": "Load testing is a process of adding heavy objects to a server to test durability."
  },
  {
    "Question": "What is regression in ML?",
    "Answer": "Regression is teaching a model to perform worse over time."
  },
  {
    "Question": "What is classification in ML?",
    "Answer": ""
  }
]

model_ = model.with_structured_output(ValidationOutput)
from langchain.chains.combine_documents import create_stuff_documents_chain  
from langchain_core.prompts import ChatPromptTemplate
# response = model_.invoke(query)
prompt = ChatPromptTemplate.from_template(query)
chain = prompt | model_


   

In [121]:
response =chain.invoke({"context":questions})


In [131]:
response.domain_specific_technical_knowledge_score

[domain_specific_result(domainName='security', Number_of_question=2, Number_of_answers_correct=2, Number_of_answers_incorrect=0, Number_of_skipped_questions=0),
 domain_specific_result(domainName='machine_learning', Number_of_question=7, Number_of_answers_correct=3, Number_of_answers_incorrect=3, Number_of_skipped_questions=1),
 domain_specific_result(domainName='backend', Number_of_question=6, Number_of_answers_correct=2, Number_of_answers_incorrect=3, Number_of_skipped_questions=1),
 domain_specific_result(domainName='networking', Number_of_question=3, Number_of_answers_correct=1, Number_of_answers_incorrect=2, Number_of_skipped_questions=0),
 domain_specific_result(domainName='general_computing', Number_of_question=5, Number_of_answers_correct=1, Number_of_answers_incorrect=4, Number_of_skipped_questions=0),
 domain_specific_result(domainName='databases', Number_of_question=3, Number_of_answers_correct=0, Number_of_answers_incorrect=3, Number_of_skipped_questions=0),
 domain_specifi

In [130]:
response.domain_specific_technical_knowledge_score.keys()

AttributeError: 'list' object has no attribute 'keys'

In [ ]:
def give_links_multi_engine(topics: list):
    qw = {}
    for q in topics:
        query = q.topic + " " + q.subconcept
        search_query = quote_plus(query)
        result = []
        
        # Try Startpage first
        try:
            url = f"https://www.startpage.com/sp/search?query={search_query}"
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"}
            resp = req.get(url, headers=headers)
            from bs4 import BeautifulSoup
            soup = BeautifulSoup(resp.text, "html.parser")
            for a in soup.select("a.w-gl__result-title"):
                if len(result) == 4:
                    break
                href = a.get("href")
                if href and href.startswith("http"):
                    result.append(href)
        except:
            pass
            
        # If no results, try Yahoo
        if not result:
            try:
                url = f"https://search.yahoo.com/search?p={search_query}"
                headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"}
                resp = req.get(url, headers=headers)
                from bs4 import BeautifulSoup
                soup = BeautifulSoup(resp.text, "html.parser")
                for a in soup.select("h3.title a"):
                    if len(result) == 4:
                        break
                    href = a.get("href")
                    if href and href.startswith("http"):
                        result.append(href)
            except:
                pass
                
        qw[query] = {"links": result, "qa": [q.question, q.wrong_answer]}
    return qw


In [14]:
qa = response.wrong_qa

In [15]:
qa[0].topic+" "+ qa[0].subconcept

'machine learning batch'

In [16]:
qa

[WrongQAItem(question='What is a batch in machine learning?', wrong_answer='A batch is a subset of training data processed together in one iteration of model training.', topic='machine learning', subconcept='batch'),
 WrongQAItem(question='What is overfitting in machine learning?', wrong_answer='Overfitting occurs when a model performs better on new data than on training data.', topic='machine learning', subconcept='overfitting'),
 WrongQAItem(question='What is the role of a load balancer in backend architecture?', wrong_answer='A load balancer distributes incoming traffic to a single server to improve latency.', topic='backend architecture', subconcept='load balancer'),
 WrongQAItem(question='What is the difference between supervised and unsupervised learning?', wrong_answer='In supervised learning, data is unlabeled, while in unsupervised learning, data is labeled.', topic='machine learning', subconcept='supervised and unsupervised learning'),
 WrongQAItem(question='What is gradient 

In [33]:
kol = give_links_multi_engine(qa)


In [53]:
len(kol.keys())

41

In [34]:
kol

{'machine learning batch': {'links': ['https://r.search.yahoo.com/_ylt=AwrO62h2G5hoDd8JdMdXNyoA;_ylu=Y29sbwNncTEEcG9zAzIEdnRpZAMEc2VjA3Nj/RV=2/RE=1756008566/RO=10/RU=https%3a%2f%2fwww.clrn.org%2fwhat-is-batch-size-in-machine-learning%2f/RK=2/RS=iAG_Szq9ERuJuQj1serO0u8n4js-',
   'https://r.search.yahoo.com/_ylt=AwrO62h2G5hoDd8JdsdXNyoA;_ylu=Y29sbwNncTEEcG9zAzIEdnRpZAMEc2VjA3Nj/RV=2/RE=1756008566/RO=10/RU=https%3a%2f%2fwiki.cloudfactory.com%2fdocs%2fmp-wiki%2ftraining-parameters%2feverything-you-need-to-know-about-batches-in-machine-learning/RK=2/RS=4OIctwVOpYddLKCiWTbed4eMT0E-',
   'https://r.search.yahoo.com/_ylt=AwrO62h2G5hoDd8JeMdXNyoA;_ylu=Y29sbwNncTEEcG9zAzIEdnRpZAMEc2VjA3Nj/RV=2/RE=1756008566/RO=10/RU=https%3a%2f%2fwww.clrn.org%2fwhat-is-batch-size-in-machine-learning%2f/RK=2/RS=iAG_Szq9ERuJuQj1serO0u8n4js-',
   'https://r.search.yahoo.com/_ylt=AwrO62h2G5hoDd8JesdXNyoA;_ylu=Y29sbwNncTEEcG9zAzIEdnRpZAMEc2VjA3Nj/RV=2/RE=1756008566/RO=10/RU=https%3a%2f%2fmedium.com%2fanalytics-vidhya